<a href="https://colab.research.google.com/github/bmnds/uea-data-science-04-data-processing/blob/main/Atividade_Pratica_Aula1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Instalação e configuração do banco de dados PostgreSQL

In [ ]:
# Pacotes necessários
import pandas as pd
from sqlalchemy import create_engine
from os import environ as env

In [ ]:
# Definindo as variávies de ambiente
from os import environ as env

%env DB_NAME=northwind
%env DB_USR=postgres
%env DB_PWD=postgres
%env DB_HOST=localhost
%env DB_PORT=5432

env: DB_NAME=northwind
env: DB_USR=postgres
env: DB_PWD=postgres
env: DB_HOST=localhost
env: DB_PORT=5432


In [ ]:
# Instalando e iniciando o PostgreSQL
# PODE DEMORAR ALGUNS MINUTOS
%%time
!sudo apt-get -y -qq update
!sudo apt-get -y -qq install postgresql
!sudo service postgresql start

debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 11.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package cron.
(Reading database ... 144793 files and directories currently installed.)
Preparing to unpack .../00-cron_3.0pl1-128.1ubuntu1_amd64.deb ...
Unpacking cron (3.0pl1-128.1ubuntu1) ...
Selecting previously unselected package logrotate.
Preparing to unpack .../01-logrotate_3.11.0-0.1ubuntu1_amd64.deb ...
Unpacking logrotate (3.11.0-0.1ubuntu1) ...
Selecting previously unselected package netbase.
Preparing to unpack .../02-netbase_5.4_all.deb ...
Unpacking netbase (5.4) ...
Preparing to unpack .../03-libpq

In [ ]:
# Criando o banco de dados
!sudo -u $DB_USR psql -U $DB_USR -c "ALTER USER $DB_USR PASSWORD '$DB_PWD';"

!sudo -u $DB_USR psql -c "DROP DATABASE IF EXISTS $DB_NAME;"
!sudo -u $DB_USR psql -c "CREATE DATABASE $DB_NAME;"

print(f'Banco de dados criado com o nome `{env["DB_NAME"]}`')

ALTER ROLE
NOTICE:  database "northwind" does not exist, skipping
DROP DATABASE
CREATE DATABASE
Banco de dados criado com o nome `northwind`


In [ ]:
# Rodando o script para inicialização do banco de dados (DDL + DML)
# PODE DEMORAR ALGUNS MINUTOS
%%time
!curl -o northwind.sql -s -OL 'https://drive.google.com/uc?export=download&id=10kOWwu6Xf8g8X8iW-p9rOu7qCTFl1KIO'
!sudo -u $DB_USR psql -d $DB_NAME -q -f northwind.sql

print(f'Arquivo `northwind.sql` baixado e executado com sucesso!')

Arquivo `northwind.sql` baixado e executado com sucesso!
CPU times: user 27.2 ms, sys: 11 ms, total: 38.1 ms
Wall time: 9.06 s


In [ ]:
# Conectando com o banco de dados PostgreSQL
db_string = f'postgresql://{env["DB_USR"]}:{env["DB_PWD"]}@{env["DB_HOST"]}:{env["DB_PORT"]}/{env["DB_NAME"]}'
db = create_engine(db_string)

print(pd.read_sql_query("SELECT 'Olá, Postgres!' AS TESTE_DE_CONEXAO;",con=db).head())
print(f'Conexão com o banco de dados estabelecida com sucesso!')
db.dispose()

  teste_de_conexao
0   Olá, Postgres!
Conexão com o banco de dados estabelecida com sucesso!


/usr/local/lib/python3.6/dist-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


# Consultando o banco de dados PostgreSQL

In [ ]:
# funções utilitárias
def print_number_of_rows(df):
  rows, cols = df.shape
  print(f'A consulta retornou {rows} registros')

In [ ]:
# 1. Quais os 10 produtos mais caros?
sql = """
  SELECT DISTINCT products.product_name, products.unit_price 
  FROM products 
  ORDER BY products.unit_price DESC
  LIMIT 10
"""
df = pd.read_sql_query(sql, con=db)
df

,product_name,unit_price
0,Côte de Blaye,263.50
1,Thüringer Rostbratwurst,123.79
2,Mishi Kobe Niku,97.00
3,Sir Rodney's Marmalade,81.00
4,Carnarvon Tigers,62.50
5,Raclette Courdavault,55.00
6,Manjimup Dried Apples,53.00
7,Tarte au sucre,49.30
8,Ipoh Coffee,46.00
9,Rössle Sauerkraut,45.60


In [ ]:
# 2. Quais clientes fizeram pedidos no último trimestre do ano de 1996?
sql = """
  SELECT DISTINCT customers.customer_id, customers.company_name, customers.city, customers.country
  FROM customers 
  RIGHT JOIN orders 
    ON customers.customer_id = orders.customer_id
  WHERE orders.order_date BETWEEN date'1996-10-01' AND date'1996-12-31';
"""

df = pd.read_sql_query(sql, con=db)
print_number_of_rows(df)
df.head()

A consulta retornou 45 registros


,customer_id,company_name,city,country
0,ANTON,Antonio Moreno Taquería,México D.F.,Mexico
1,AROUT,Around the Horn,London,UK
2,BERGS,Berglunds snabbköp,Luleå,Sweden
3,BLONP,Blondesddsl père et fils,Strasbourg,France
4,BOLID,Bólido Comidas preparadas,Madrid,Spain


In [ ]:
# 3. Cálculo do subtotal para cada produto incluído no pedido
sql = """
  SELECT DISTINCT 
      order_details.order_id, 
      order_details.product_id,
      products.product_name, 
      order_details.unit_price, 
      order_details.quantity,
      order_details.discount, 
      round(CAST(unit_price * quantity * (1-discount) AS NUMERIC),2) AS product_subtotal
  FROM products 
  NATURAL JOIN order_details
  ORDER BY order_details.order_id;
"""

df = pd.read_sql_query(sql, con=db)
print_number_of_rows(df)
df.head()

A consulta retornou 1493 registros


,order_id,product_id,product_name,unit_price,quantity,discount,product_subtotal
0,10248,72,Mozzarella di Giovanni,34.8,5,0.0,174.0
1,10498,24,Guaraná Fantástica,4.5,14,0.0,63.0
2,10498,40,Boston Crab Meat,18.4,5,0.0,92.0
3,10498,42,Singaporean Hokkien Fried Mee,14.0,30,0.0,420.0
4,10499,28,Rössle Sauerkraut,45.6,20,0.0,912.0


In [ ]:
# 4. Produtos com preço abaixo do valor médio

sql = """
  SELECT products.product_name, products.unit_price
  FROM products
  WHERE products.unit_price < (SELECT AVG(unit_price) FROM products)
  ORDER BY products.unit_price DESC
"""

df = pd.read_sql_query(sql, con=db)
print_number_of_rows(df)
df.head()

A consulta retornou 52 registros


,product_name,unit_price
0,Sirop d'érable,28.50
1,Gravad lax,26.00
2,Nord-Ost Matjeshering,25.89
3,Grandma's Boysenberry Spread,25.00
4,Pâté chinois,24.00


In [ ]:
# 5. Cálculo do valor total do pedido
sql = """
  SELECT order_id, sum(round(CAST(unit_price * quantity * (1-discount) AS NUMERIC),2)) AS order_total
  FROM order_details
  GROUP BY order_id
"""

df = pd.read_sql_query(sql, con=db)
print_number_of_rows(df)
df.head()

A consulta retornou 830 registros


,order_id,order_total
0,11038,732.60
1,10782,12.50
2,10725,287.80
3,10423,1020.00
4,10518,4150.05


In [ ]:
# 6. Vendas por ano (trabalho em equipe)

In [ ]:
# 7. Produtos vendidos em 1995 (trabalho em equipe)

In [ ]:
# 8. Vendas por Categoria (trabalho em equipe)